#Preparation
1. Get gmail account (for GoogleDrive)
2. Get ngrok account from http://www.ngrok.com
3. Paste content of ~/.ssh/id_rsa.pub below. If you have no this file, you can:
    1. Generate it by ssh-gen in Ubuntu or Mac OS
    2. Use PuttyGen in windows ( https://www.putty.org/ )
4. Paste authtoken from ngrok below
5. Run this notebook
6. Copy ssh command with ngrok tunnel port and address (look at the end of this notebook). Or use Putty with correspondent address and port

#Mount your google drive & symlink it to `/drive`

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh

#(optional) If a TPU is attached, print its address

This notebook works for all runtime types: CPU, GPU, and TPU.

If connected to TPU runtime, this cell prints info about the attached TPU. (This is just for convenience / informational purposes.)

In [ ]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('Not connected to a TPU runtime')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)
  with tf.compat.v1.Session(tpu_address) as session:
    devices = session.list_devices()
  print('TPU devices:')
  pprint.pprint(devices)

# Install `sshd`, `tmux`, and `vim`

You can add any other prerequisites you want, e.g. `emacs`. That way, when you ssh into the colab instance, all of these will already be installed.

(You'd otherwise have to run `apt-get install emacs` every time you ssh into a fresh colab instance. It's easier just to install all your favorite programs here, ahead of time.)

In [ ]:
!apt-get install ssh tmux nano

# Create an sshd configuration file

- Normally, you shouldn't need to modify this next cell; the configuration is designed to work with colab.

Simply run this cell:

In [ ]:
#@title
%%writefile /drive/ngrok-ssh/sshd_config
#	$OpenBSD: sshd_config,v 1.101 2017/03/14 07:19:07 djm Exp $

# This is the sshd server system-wide configuration file.  See
# sshd_config(5) for more information.

# This sshd was compiled with PATH=/usr/bin:/bin:/usr/sbin:/sbin

# The strategy used for options in the default sshd_config shipped with
# OpenSSH is to specify options with their default value where
# possible, but leave them commented.  Uncommented options override the
# default value.

#Port 22
#AddressFamily any
#ListenAddress 0.0.0.0
#ListenAddress ::

#HostKey /etc/ssh/ssh_host_rsa_key
#HostKey /etc/ssh/ssh_host_ecdsa_key
#HostKey /etc/ssh/ssh_host_ed25519_key

# Ciphers and keying
#RekeyLimit default none

# Logging
#SyslogFacility AUTH
#LogLevel INFO

# Authentication:

#LoginGraceTime 2m
#PermitRootLogin prohibit-password
#StrictModes yes
#MaxAuthTries 6
#MaxSessions 10

#PubkeyAuthentication yes

# Expect .ssh/authorized_keys2 to be disregarded by default in future.
#AuthorizedKeysFile	.ssh/authorized_keys .ssh/authorized_keys2

#AuthorizedPrincipalsFile none

#AuthorizedKeysCommand none
#AuthorizedKeysCommandUser nobody

# For this to work you will also need host keys in /etc/ssh/ssh_known_hosts
#HostbasedAuthentication no
# Change to yes if you don't trust ~/.ssh/known_hosts for
# HostbasedAuthentication
#IgnoreUserKnownHosts no
# Don't read the user's ~/.rhosts and ~/.shosts files
#IgnoreRhosts yes

# To disable tunneled clear text passwords, change to no here!
#PasswordAuthentication yes
#PermitEmptyPasswords no

# Change to yes to enable challenge-response passwords (beware issues with
# some PAM modules and threads)
ChallengeResponseAuthentication no

# Kerberos options
#KerberosAuthentication no
#KerberosOrLocalPasswd yes
#KerberosTicketCleanup yes
#KerberosGetAFSToken no

# GSSAPI options
#GSSAPIAuthentication no
#GSSAPICleanupCredentials yes
#GSSAPIStrictAcceptorCheck yes
#GSSAPIKeyExchange no

# Set this to 'yes' to enable PAM authentication, account processing,
# and session processing. If this is enabled, PAM authentication will
# be allowed through the ChallengeResponseAuthentication and
# PasswordAuthentication.  Depending on your PAM configuration,
# PAM authentication via ChallengeResponseAuthentication may bypass
# the setting of "PermitRootLogin without-password".
# If you just want the PAM account and session checks to run without
# PAM authentication, then enable this but set PasswordAuthentication
# and ChallengeResponseAuthentication to 'no'.
UsePAM yes

#AllowAgentForwarding yes
AllowTcpForwarding yes
#GatewayPorts no
X11Forwarding yes
#X11DisplayOffset 10
#X11UseLocalhost yes
#PermitTTY yes
PrintMotd no
#PrintLastLog yes
#TCPKeepAlive yes
#UseLogin no
#PermitUserEnvironment no
#Compression delayed
#ClientAliveInterval 0
#ClientAliveCountMax 3
#UseDNS no
#PidFile /var/run/sshd.pid
#MaxStartups 10:30:100
#PermitTunnel no
#ChrootDirectory none
#VersionAddendum none

# no default banner path
#Banner none

# Allow client to pass locale environment variables
AcceptEnv LANG LC_*

# override default of no subsystems
Subsystem	sftp	/usr/lib/openssh/sftp-server

# Example of overriding settings on a per-user basis
#Match User anoncvs
#	X11Forwarding no
#	AllowTcpForwarding no
#	PermitTTY no
#	ForceCommand cvs server

# Paste the contents of your local `~/.ssh/id_rsa.pub` file into this next cell

- If you don't have an `~/.ssh/id_rsa.pub` file, use `ssh-keygen` to make one.

- Remember to actually run the cell after you paste your key into it!



In [ ]:
%%writefile ~/.ssh/authorized_keys
#<paste content of id_rsa.pub or PuttyGen output here>

# Restart sshd

In [ ]:
!cp /drive/ngrok-ssh/sshd_config /etc/ssh/sshd_config
!service ssh restart
!mkdir -p ~/.ssh

#Install ngrok

In [ ]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

# Sign-in with ngrok

- Sign up for a free ngrok account: https://ngrok.com

- Copy your authtoken from https://dashboard.ngrok.com/auth and paste it into the following cell

- Run the cell

In [ ]:
!/ngrok authtoken #<paste authtoken from you ngrok here>

# Create various ngrok tunnel configurations

Run each of these cells, one by one:

In [ ]:
%%writefile /drive/ngrok-ssh/ssh.yml
tunnels:
  ssh:
    proto: tcp
    addr: 22

In [ ]:
%%writefile /drive/ngrok-ssh/vnc.yml
tunnels:
  vnc:
    proto: tcp
    addr: 5920

In [ ]:
%%writefile /drive/ngrok-ssh/tensorboard.yml
tunnels:
  tensorboard:
    proto: http
    addr: 6006
    inspect: false
    bind_tls: true

In [ ]:
%%writefile /drive/ngrok-ssh/http8080.yml
tunnels:
  http8080:
    proto: http
    addr: 8080
    inspect: false
    bind_tls: true

In [ ]:
%%writefile /drive/ngrok-ssh/tcp8080.yml
tunnels:
  tcp8080:
    proto: tcp
    addr: 8080

In [ ]:
import os
if 'COLAB_TPU_ADDR' in os.environ:
  with open('/drive/ngrok-ssh/tpu.yml', 'w') as f:
    f.write("""
tunnels:
  tpu:
    proto: tcp
    addr: {}
""".format(os.environ['COLAB_TPU_ADDR']).strip())
  print('Wrote /drive/ngrok-ssh/tpu.yml')

In [ ]:
%%writefile /drive/ngrok-ssh/run_ngrok.sh
#!/bin/sh
set -x
/ngrok start --config ~/.ngrok2/ngrok.yml --config /drive/ngrok-ssh/ssh.yml --config /drive/ngrok-ssh/vnc.yml --log=stdout --config /drive/ngrok-ssh/tensorboard.yml --config /drive/ngrok-ssh/http8080.yml --config /drive/ngrok-ssh/tcp8080.yml --config /drive/ngrok-ssh/tpu.yml "$@"

#Start ngrok

In [ ]:
import os
if 'COLAB_TPU_ADDR' in os.environ:
  get_ipython().system_raw('sh /drive/ngrok-ssh/run_ngrok.sh ssh tensorboard tcp8080 tpu &')
else:
  get_ipython().system_raw('sh /drive/ngrok-ssh/run_ngrok.sh ssh tensorboard tcp8080 &')

In [ ]:
!ps aux | grep ngrok

# Make an ngrok API
Let's define some helper functions:

- `get_ngrok_info()` - returns a JSON object containing info about all open ngrok tunnels

- `get_ngrok_tunnels()` - enumerate over all open ngrok tunnels. Yields `name, info` tuples.

- `get_ngrok_tunnel(name)` - get info about a tunnel

- `get_ngrok_url(name, local=False)` - get a tunnel's URL. If `local` is `False`, returns a public URL that anyone can access, otherwise returns the local url being forwarded

In [ ]:
import requests
import urllib.parse

def get_ngrok_info():
  return requests.get('http://localhost:4040/api/tunnels').json()

def get_ngrok_tunnels():
  for tunnel in get_ngrok_info()['tunnels']:
    name = tunnel['name']
    yield name, tunnel

def get_ngrok_tunnel(name):
  for name1, tunnel in get_ngrok_tunnels():
    if name == name1:
      return tunnel

def get_ngrok_url(name, local=False):
  if local:
    return get_ngrok_tunnel(name)['config']['addr']
  else:
    return get_ngrok_tunnel(name)['public_url']

# Skip this section

These are various commands for debugging ngrok.

In [ ]:
#!bash /drive/ngrok-ssh/run_ngrok.sh ssh vnc tensorboard tcp8080 tpu

In [ ]:
#!curl -s http://localhost:4040/api/tunnels  | python3 -c "import sys, json; url=json.load(sys.stdin)['tunnels'][0]['public_url']; import urllib.parse; print(urllib.parse.urlparse(url).netloc)"

#Start Tensorboard

In [ ]:
LOG_DIR = '/drive/gpt-2-train/checkpoint/run1'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)


In [ ]:
!ps aux | grep tensorboard

# How to view tensorboard

In [ ]:
print('To view tensorboard, visit', get_ngrok_url('tensorboard'))

# Display each ngrok URL

For each ngrok tunnel, this cell prints its name, its public URL, and the local URL being forwarded to that public URL.

In [ ]:
for name, tunnel in get_ngrok_tunnels():
  local_url = get_ngrok_url(name, local=True)
  public_url = get_ngrok_url(name, local=False)
  print('{:12s} {} <-> {}'.format(name, public_url, local_url))

# How to SSH into this Colab instance

In [ ]:
print('')
public_url, public_port = urllib.parse.urlparse(get_ngrok_url('ssh')).netloc.split(':')
print('To SSH into this colab instance, run the following command on your local machine:')
print('')

print('  ssh -p {} root@{}'.format(public_port, public_url))